# Player Trueskill

### Helpful Resources
[Trueskill Documentation](https://trueskill.org/#version-0-4-4)  
[Trueskill for DOTA2 Players - Kaggle](https://www.kaggle.com/devinanzelmo/dota-2-skill-rating-with-trueskill)  
[Collection of Trueskill Examples](https://python.hotexamples.com/examples/trueskill/TrueSkill/-/python-trueskill-class-examples.html)  

In [1]:
# Housekeeping
import requests
import datetime
import pandas as pd
import numpy as np
import io
import os
import trueskill
import itertools
import math

import oeutils

pd.options.display.max_rows = 100
pd.options.display.max_columns = 500

workingdir = '../data/'
years = ['2018', '2019', '2020', '2021']

#oeutils.download(workingdir, years)
df = oeutils.read(workingdir, years)
df = oeutils.clean(df, split_on='player', keep_identities=False, keep_leagues=False, keep_columns=False)

/home/paulperrone/source/lol-analysis/notebooks/oeutils.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oe_data['gameid'] = oe_data['gameid'].str.strip()


In [2]:
df.shape

(188865, 117)

In [3]:
df['league'].unique()

array(['LPL', 'LCK', 'CK', 'VCS', 'LCS.A', 'EU LCS', 'LLN', 'OPL', 'LMS',
       'TCL', 'CBLOL', 'CLS', 'NA LCS', 'TCS', 'LJL', 'LCL', 'CIS CL',
       'BRCC', 'GPL', 'EM', 'MSI', 'OCS', 'WCS', 'UL', 'NASG', 'DC',
       'KeSPA', 'LEC', 'LLA', 'TRA', 'BIG', 'LFL', 'LCS', 'UKLC', 'LDL',
       'UPL', 'PCS', 'Riot', 'MSC', 'RCL', 'CU', 'NLC', 'NEST', 'BL',
       'PGN', 'LCK CL', 'GLL', 'SL', 'PRM', 'CBLOL.A', 'EBL', 'BM'],
      dtype=object)

In [4]:
df.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
348,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,1,Blue,top,Duke,Invictus Gaming,Ornn,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,1,3,12,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9631,363.4340,0.150020,315.4340,NaN,13,0.4906,6,0.2264,2,NaN,NaN,10791,7272,274.4151,0.188550,8870.0,NaN,218.0,216.0,2,1,0,8.2264,3292.0,4758.0,90.0,3287.0,4640.0,88.0,5.0,118.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5280.0,7775.0,150.0,5060.0,7477.0,144.0,220.0,298.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
349,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,2,Blue,jng,Ning,Invictus Gaming,Kha'Zix,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,0,5,12,4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6924,261.2830,0.107854,693.8113,NaN,48,1.8113,24,0.9057,10,NaN,NaN,10831,7312,275.9245,0.189587,9775.0,NaN,194.0,36.0,158,96,8,7.3208,3170.0,3745.0,67.0,2988.0,3386.0,58.0,182.0,359.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4660.0,5588.0,100.0,4290.0,4957.0,80.0,370.0,631.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
350,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,3,Blue,mid,Rookie,Invictus Gaming,Orianna,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,1,8,12,4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19985,754.1509,0.311303,274.6415,NaN,10,0.3774,7,0.2642,5,NaN,NaN,12165,8646,326.2642,0.224176,9310.0,NaN,266.0,246.0,20,16,4,10.0377,3633.0,5236.0,108.0,3378.0,5108.0,92.0,255.0,128.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,5389.0,7541.0,157.0,5211.0,8079.0,150.0,178.0,-538.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
351,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,4,Blue,bot,JackeyLove,Invictus Gaming,Ezreal,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,1,7,12,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23623,891.4340,0.367971,321.2075,NaN,14,0.5283,6,0.2264,2,NaN,NaN,14028,10509,396.5660,0.272482,13075.0,NaN,276.0,257.0,19,8,11,10.4151,3992.0,3286.0,97.0,3438.0,3018.0,88.0,554.0,268.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,6800.0,5569.0,159.0,5138.0,5263.0,134.0,1662.0,306.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
352,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,5,Blue,sup,Baolan,Invictus Gaming,Tahm Kench,A

In [5]:
lcs = df[df['league'].isin(['LCS', 'NA LCS'])]
lcs.shape

(7590, 117)

In [7]:
lcs = lcs.sort_values(by=['date', 'champion'])

In [9]:
lcs.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
1701,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,10,Red,sup,Mithy,TSM,Alistar,Kog'Maw,Tahm Kench,Ezreal,Braum,Shen,1687,0,0,3,1,1,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0356,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,2582,91.8317,0.093656,389.9822,516.3841,36,1.2804,10,0.3557,10,63.0,2.2407,7024,3307,117.6171,0.116299,5700.0,NaN,78.0,78.0,0,0,0,2.7742,2263.0,2564.0,26.0,2191.0,2484.0,25.0,72.0,80.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3453.0,3812.0,41.0,3930.0,4362.0,45.0,-477.0,-550.0,-4.0,0.0,0.0,1.0,0.0,3.0,0.0
1692,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,1,Blue,top,Impact,Team Liquid,Gangplank,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,3,0,6,11,1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,16858,599.5732,0.367870,386.8880,319.9881,14,0.4979,2,0.0711,2,24.0,0.8536,13474,9757,347.0184,0.239742,12133.0,NaN,254.0,248.0,6,1,4,9.0338,3650.0,4637.0,77.0,3119.0,4254.0,78.0,531.0,383.0,-1.0,0.0,1.0,0.0,0.0,0.0,1.0,5824.0,7885.0,130.0,5087.0,7378.0,140.0,737.0,507.0,-10.0,0.0,3.0,0.0,0.0,0.0,1.0
1697,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,6,Red,top,Hauntzer,TSM,Gnar,Kog'Maw,Tahm Kench,Ezreal,Braum,Shen,1687,0,0,3,0,1,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0356,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,7391,262.8690,0.268091,576.0640,422.3829,15,0.5335,1,0.0356,3,26.0,0.9247,10105,6388,227.1962,0.224653,9075.0,NaN,271.0,271.0,0,0,0,9.6384,3119.0,4254.0,78.0,3650.0,4637.0,77.0,-531.0,-383.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,5087.0,7378.0,140.0,5824.0,7885.0,130.0,-737.0,-507.0,10.0,0.0,0.0,1.0,0.0,3.0,0.0
1693,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,2,Blue,jng,Xmithie,Team Liquid,Gragas,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,0,0,9,11,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,4355,154.8903,0.095033,778.6841,808.5951,22,0.7825,9,0.3201,9,56.0,1.9917,9998,6281,223.3906,0.154331,8325.0,NaN,162.0,26.0,136,97,16,5.7617,2801.0,3194.0,50.0,3190.0,3707.0,70.0,-389.0,-513.0,-20.0,0.0,0.0,0.0,0.0,0.0,0.0,4641.0,5945.0,88.0,4643.0,5427.0,101.0,-2.0,518.0,-13.0,0.0,3.0,0.0,0.0,0.0,1.0
1700,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-

In [10]:
input_data = lcs[['gameid', 'date', 'champion', 'side', 'team', 'position', 'result']]
input_data.head()

,gameid,date,champion,side,team,position,result
1701,TRLH1/1002440062,2018-01-20 20:50:09,Alistar,Red,TSM,sup,0
1692,TRLH1/1002440062,2018-01-20 20:50:09,Gangplank,Blue,Team Liquid,top,1
1697,TRLH1/1002440062,2018-01-20 20:50:09,Gnar,Red,TSM,top,0
1693,TRLH1/1002440062,2018-01-20 20:50:09,Gragas,Blue,Team Liquid,jng,1
1700,TRLH1/1002440062,2018-01-20 20:50:09,Kalista,Red,TSM,bot,0


In [11]:
columns = ['gameid', 'date', \
           'blue_team', \
           'blue_top_name', \
           'blue_jng_name', \
           'blue_mid_name', \
           'blue_bot_name', \
           'blue_sup_name',\
           'blue_team_result', \
           'red_team', \
           'red_top_name', \
           'red_jng_name', \
           'red_mid_name', \
           'red_bot_name', \
           'red_sup_name']

lcs_rating = pd.DataFrame(columns=columns)

## Trueskill

In [12]:
ts = trueskill.TrueSkill(draw_probability=0.0)

In [13]:
champion_ratings_dict = dict()
for i in input_data['champion'].unique():
    champion_ratings_dict[i] = ts.create_rating()
    
print(champion_ratings_dict)

{'Alistar': trueskill.Rating(mu=25.000, sigma=8.333), 'Gangplank': trueskill.Rating(mu=25.000, sigma=8.333), 'Gnar': trueskill.Rating(mu=25.000, sigma=8.333), 'Gragas': trueskill.Rating(mu=25.000, sigma=8.333), 'Kalista': trueskill.Rating(mu=25.000, sigma=8.333), 'Malzahar': trueskill.Rating(mu=25.000, sigma=8.333), 'Ryze': trueskill.Rating(mu=25.000, sigma=8.333), 'Shyvana': trueskill.Rating(mu=25.000, sigma=8.333), 'Taric': trueskill.Rating(mu=25.000, sigma=8.333), 'Tristana': trueskill.Rating(mu=25.000, sigma=8.333), 'Braum': trueskill.Rating(mu=25.000, sigma=8.333), "Cho'Gath": trueskill.Rating(mu=25.000, sigma=8.333), 'Ezreal': trueskill.Rating(mu=25.000, sigma=8.333), "Kog'Maw": trueskill.Rating(mu=25.000, sigma=8.333), 'Morgana': trueskill.Rating(mu=25.000, sigma=8.333), 'Rengar': trueskill.Rating(mu=25.000, sigma=8.333), 'Sejuani': trueskill.Rating(mu=25.000, sigma=8.333), 'Shen': trueskill.Rating(mu=25.000, sigma=8.333), 'Azir': trueskill.Rating(mu=25.000, sigma=8.333), 'Evely

In [14]:
### Lambda Apply w Position as Key
def setup_match(df, gameid, position):
    #blue players
    game_frame = df[df.gameid == gameid]
    blue_data = game_frame[game_frame.side == 'Blue']
    red_data = game_frame[game_frame.side == 'Red']

    blue_top_name = blue_data[blue_data.position == 'top'].champion.iloc[0]
    blue_jng_name = blue_data[blue_data.position == 'jng'].champion.iloc[0]
    blue_mid_name = blue_data[blue_data.position == 'mid'].champion.iloc[0]
    blue_bot_name = blue_data[blue_data.position == 'bot'].champion.iloc[0]
    blue_sup_name = blue_data[blue_data.position == 'sup'].champion.iloc[0]

    # red players
    red_top_name = red_data[red_data.position == 'top'].champion.iloc[0]
    red_jng_name = red_data[red_data.position == 'jng'].champion.iloc[0]
    red_mid_name = red_data[red_data.position == 'mid'].champion.iloc[0]
    red_bot_name = red_data[red_data.position == 'bot'].champion.iloc[0]
    red_sup_name = red_data[red_data.position == 'sup'].champion.iloc[0]

    # date, teams, result
    date = blue_data['date'].iloc[0]
    blue_team_result = blue_data['result'].iloc[0]
    blue_team = blue_data['team'].iloc[0]
    red_team = red_data['team'].iloc[0]
    # create pandas series for df
    match = pd.Series([gameid, date, \
                       blue_team, \
                       blue_top_name, \
                       blue_jng_name, \
                       blue_mid_name, \
                       blue_bot_name, \
                       blue_sup_name,\
                       blue_team_result, \
                       red_team, \
                       red_top_name, \
                       red_jng_name, \
                       red_mid_name, \
                       red_bot_name, \
                       red_sup_name])
    return match


lcs_rating[['gameid', 'date', \
           'blue_team', \
           'blue_top_name', \
           'blue_jng_name', \
           'blue_mid_name', \
           'blue_bot_name', \
           'blue_sup_name',\
           'blue_team_result', \
           'red_team', \
           'red_top_name', \
           'red_jng_name', \
           'red_mid_name', \
           'red_bot_name', \
           'red_sup_name']] = input_data.apply(lambda row: setup_match(input_data, row['gameid'], row['position']),axis=1)


In [15]:
lcs_rating.shape


(7590, 15)

In [16]:
lcs_rating = lcs_rating.reset_index(drop=True)
lcs_rating = lcs_rating.drop_duplicates(subset=['gameid'])
lcs_rating = lcs_rating.sort_values(by=['date'])

In [17]:
lcs_rating.head(20)

,gameid,date,blue_team,blue_top_name,blue_jng_name,blue_mid_name,blue_bot_name,blue_sup_name,blue_team_result,red_team,red_top_name,red_jng_name,red_mid_name,red_bot_name,red_sup_name
0,TRLH1/1002440062,2018-01-20 20:50:09,Team Liquid,Gangplank,Gragas,Malzahar,Tristana,Taric,1,TSM,Gnar,Shyvana,Ryze,Kalista,Alistar
10,TRLH1/1002440076,2018-01-20 22:38:20,100 Thieves,Shen,Sejuani,Ryze,Kog'Maw,Braum,1,OpTic Gaming,Cho'Gath,Rengar,Malzahar,Ezreal,Morgana
20,TRLH1/1002440084,2018-01-21 00:14:20,Clutch Gaming,Gangplank,Evelynn,Malzahar,Tristana,Ornn,1,Golden Guardians,Vladimir,Kha'Zix,Azir,Kog'Maw,Taric
30,TRLH1/1002440095,2018-01-21 01:20:39,Echo Fox,Lucian,Zac,Ryze,Kog'Maw,Shen,1,FlyQuest,Gangplank,Jarvan IV,Cassiopeia,Varus,Alistar
40,TRLH1/1002440106,2018-01-21 02:16:56,Counter Logic Gaming,Ornn,Rengar,Malzahar,Varus,Taric,0,Cloud9,Gangplank,Sejuani,Zilean,Tristana,Braum
50,TRLH1/1002440127,2018-01-21 21:49:32,OpTic Gaming,Ornn,Jarvan IV,Orianna,Kog'Maw,Taric,0,Team Liquid,Gnar,Sejuani,Malzahar,Kalista,Thresh
60,TRLH1/1002440132,2018-01-21 22:46:16,FlyQuest,Gnar,Zac,Azir,Kalista,Taric,1,TSM,Cho'Gath,Kha'Zix,Xerath,Kog'Maw,Tahm Kench
70,TRLH1/1002440143,2018-01-22 00:00:28,100 Thieves,Gnar,Jarvan IV,Malzahar,Ezreal,Braum,1,Counter Logic Gaming,Gangplank,Sejuani,Zilean,Caitlyn,Janna
80,TRLH1/1002440150,2018-01-22 01:00:51,Golden Guardians,Gangplank,Jarvan IV,Zoe,Varus,Taric,0,Cloud9,Kled,Evelynn,Ryze,Tristana,Tahm Kench
90,TRLH1/1002440161,2018-01-22 02:02:56,Echo Fox,Gnar,Zac,Azir,Ezreal,Taric,1,Clutch Gaming,Ornn,Sejuani,Ryze,Tristana,Braum


In [18]:
analyzed_gameids = {}

def win_probability(team1, team2, trueskill_global_env):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (trueskill_global_env.beta ** 2 ) + sum_sigma)
    return trueskill_global_env.cdf(delta_mu / denom)

def update_trueskill(rating_dict, gameid_dict, gameid, blue_team_result, \
                        blue_top_name, \
                        blue_jng_name, \
                        blue_mid_name, \
                        blue_bot_name, \
                        blue_sup_name,\
                        red_top_name, \
                        red_jng_name, \
                        red_mid_name, \
                        red_bot_name, \
                        red_sup_name):
    rating_groups = [(rating_dict[blue_top_name], \
                      rating_dict[blue_jng_name], \
                      rating_dict[blue_mid_name], \
                      rating_dict[blue_bot_name], \
                      rating_dict[blue_sup_name]), \
                     (rating_dict[red_top_name], \
                      rating_dict[red_jng_name], \
                      rating_dict[red_mid_name], \
                      rating_dict[red_bot_name], \
                      rating_dict[red_sup_name])]
    blue_mu = rating_groups[0][0].mu
    blue_sigma = rating_groups[0][0].sigma
    red_mu = rating_groups[1][0].mu
    red_sigma = rating_groups[1][0].sigma
    blue_team_win_prob = win_probability(rating_groups[0], rating_groups[1], ts)
    # Get Mu by position
    blue_top_mu = rating_dict[blue_top_name].mu
    blue_jng_mu = rating_dict[blue_jng_name].mu
    blue_mid_mu = rating_dict[blue_mid_name].mu
    blue_bot_mu = rating_dict[blue_bot_name].mu
    blue_sup_mu = rating_dict[blue_sup_name].mu
    red_top_mu = rating_dict[red_top_name].mu
    red_jng_mu = rating_dict[red_jng_name].mu
    red_mid_mu = rating_dict[red_mid_name].mu
    red_bot_mu = rating_dict[red_bot_name].mu
    red_sup_mu = rating_dict[red_sup_name].mu
    # Get Sigma by position
    blue_top_sigma = rating_dict[blue_top_name].sigma
    blue_jng_sigma = rating_dict[blue_jng_name].sigma
    blue_mid_sigma = rating_dict[blue_mid_name].sigma
    blue_bot_sigma = rating_dict[blue_bot_name].sigma
    blue_sup_sigma = rating_dict[blue_sup_name].sigma
    red_top_sigma = rating_dict[red_top_name].sigma
    red_jng_sigma = rating_dict[red_jng_name].sigma
    red_mid_sigma = rating_dict[red_mid_name].sigma
    red_bot_sigma = rating_dict[red_bot_name].sigma
    red_sup_sigma = rating_dict[red_sup_name].sigma
    # Update ratings
    if blue_team_result == 1:
        rated_rating_groups = ts.rate(rating_groups, ranks=[0, 1]) # for ranks 0 is winner 
    else:
        rated_rating_groups = ts.rate(rating_groups, ranks=[1, 0])
    # Return values for new columns
    ts_update = pd.Series([blue_team_win_prob, blue_mu, blue_sigma, red_mu, red_sigma, \
           blue_top_mu, blue_top_sigma, \
           blue_jng_mu, blue_jng_sigma, \
           blue_mid_mu, blue_mid_sigma, \
           blue_bot_mu, blue_bot_sigma, \
           blue_sup_mu, blue_sup_sigma, \
           red_top_mu, red_top_sigma, \
           red_jng_mu, red_jng_sigma, \
           red_mid_mu, red_mid_sigma, \
           red_bot_mu, red_bot_sigma, \
           red_sup_mu, red_sup_sigma])
    # update the rating dictionary
    rating_dict[blue_top_name] = rated_rating_groups[0][0]
    rating_dict[blue_jng_name] = rated_rating_groups[0][1]
    rating_dict[blue_mid_name] = rated_rating_groups[0][2]
    rating_dict[blue_bot_name] = rated_rating_groups[0][3]
    rating_dict[blue_sup_name] = rated_rating_groups[0][4]
    rating_dict[red_top_name] = rated_rating_groups[1][0]
    rating_dict[red_jng_name] = rated_rating_groups[1][1]
    rating_dict[red_mid_name] = rated_rating_groups[1][2]
    rating_dict[red_bot_name] = rated_rating_groups[1][3]
    rating_dict[red_sup_name] = rated_rating_groups[1][4]
    # determine if use old data because of repeat
    if gameid in gameid_dict:
        return gameid_dict[gameid]
    else:
        gameid_dict[gameid] = ts_update
        return ts_update

lcs_rating[['blue_team_win_prob', 'blue_mu', 'blue_sigma', 'red_mu', 'red_sigma',\
           'blue_top_mu', 'blue_top_sigma', \
           'blue_jng_mu', 'blue_jng_sigma', \
           'blue_mid_mu', 'blue_mid_sigma', \
           'blue_bot_mu', 'blue_bot_sigma', \
           'blue_sup_mu', 'blue_sup_sigma', \
           'red_top_mu', 'red_top_sigma', \
           'red_jng_mu', 'red_jng_sigma', \
           'red_mid_mu', 'red_mid_sigma', \
           'red_bot_mu', 'red_bot_sigma', \
           'red_sup_mu', 'red_sup_sigma']] = lcs_rating.apply(lambda row: update_trueskill(champion_ratings_dict, analyzed_gameids, row['gameid'], row['blue_team_result'], \
                        row['blue_top_name'], \
                        row['blue_jng_name'], \
                        row['blue_mid_name'], \
                        row['blue_bot_name'], \
                        row['blue_sup_name'],\
                        row['red_top_name'], \
                        row['red_jng_name'], \
                        row['red_mid_name'], \
                        row['red_bot_name'], \
                        row['red_sup_name']), axis=1)

In [19]:
lcs_rating.shape

(759, 40)

In [20]:
lcs_rating = lcs_rating.reset_index(drop=True)
lcs_rating.head(20)

,gameid,date,blue_team,blue_top_name,blue_jng_name,blue_mid_name,blue_bot_name,blue_sup_name,blue_team_result,red_team,red_top_name,red_jng_name,red_mid_name,red_bot_name,red_sup_name,blue_team_win_prob,blue_mu,blue_sigma,red_mu,red_sigma,blue_top_mu,blue_top_sigma,blue_jng_mu,blue_jng_sigma,blue_mid_mu,blue_mid_sigma,blue_bot_mu,blue_bot_sigma,blue_sup_mu,blue_sup_sigma,red_top_mu,red_top_sigma,red_jng_mu,red_jng_sigma,red_mid_mu,red_mid_sigma,red_bot_mu,red_bot_sigma,red_sup_mu,red_sup_sigma
0,TRLH1/1002440062,2018-01-20 20:50:09,Team Liquid,Gangplank,Gragas,Malzahar,Tristana,Taric,1,TSM,Gnar,Shyvana,Ryze,Kalista,Alistar,0.500000,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333
1,TRLH1/1002440076,2018-01-20 22:38:20,100 Thieves,Shen,Sejuani,Ryze,Kog'Maw,Braum,1,OpTic Gaming,Cho'Gath,Rengar,Malzahar,Ezreal,Morgana,0.422155,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,22.127021,7.966146,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,27.872979,7.966146,25.000000,8.333333,25.000000,8.333333
2,TRLH1/1002440084,2018-01-21 00:14:20,Clutch Gaming,Gangplank,Evelynn,Malzahar,Tristana,Ornn,1,Golden Guardians,Vladimir,Kha'Zix,Azir,Kog'Maw,Taric,0.521157,27.872979,7.966146,25.000000,8.333333,27.872979,7.966146,25.000000,8.333333,25.862365,7.764037,27.872979,7.966146,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,27.200217,8.101604,27.872979,7.966146
3,TRLH1/1002440095,2018-01-21 01:20:39,Echo Fox,Lucian,Zac,Ryze,Kog'Maw,Shen,1,FlyQuest,Gangplank,Jarvan IV,Cassiopeia,Varus,Alistar,0.501757,25.000000,8.333333,29.549701,7.775597,25.000000,8.333333,25.000000,8.333333,24.137635,7.764037,25.465994,7.901054,27.200217,8.101604,29.549701,7.775597,25.000000,8.333333,25.000000,8.333333,25.000000,8.333333,22.127021,7.966146
4,TRLH1/1002440106,2018-01-21 02:16:56,Counter Logic Gaming,Ornn,Rengar,Malzahar,Varus,Taric,0,Cloud9,Gangplank,Sejuani,Zilean,Tristana,Braum,0.356405,26.834839,8.114868,27.884126,7.594560,26.834839,8.114868,22.799783,8.101604,27.455095,7.587777,23.086943,8.109953,26.196257,7.775597,27.884126,7.594560,27.200217,8.101604,25.000000,8.333333,29.549701,7.775597,27.200217,8.101604
5,TRLH1/1002440127,2018-01-21 21:49:32,OpTic Gaming,Ornn,Jarvan IV,Orianna,Kog'Maw,Taric,0,Team Liquid,Gnar,Sejuani,Malzahar,Kalista,Thresh,0.523285,25.492589,7.931686,22.127021,7.966146,25.492589,7.931686,23.086943,8.109953,25.000000,8.333333,27.185744,7.711010,24.963885,7.614664,22.127021,7.966146,28.538083,7.919328,26.281533,7.438344,22.127021,7.966146,25.000000,8.333333
6,TRLH1/1002440132,2018-01-21 22:46:16,FlyQuest,Gnar,Zac,Azir,Kalista,Taric,1,TSM,Cho'Gath,Kha'Zix,Xerath,Kog'Maw,Tahm Kench,0.499012,23.997350,7.759692,22.799783,8.101604,23.997350,7.759692,26.913057,8.109953,23.165161,8.114868,23.997350,7.759692,23.254942,7.434629,22.799783,8.101604,23.165161,8.114868,25.000000,8.333333,25.433288,7.523975,25.000000,8.333333
7,TRLH1/1002440143,2018-01-22 00:00:28,100 Thieves,Gnar,Jarvan IV,Malzahar,Ezreal,Braum,1,Counter Logic Gaming,Gangplank,Sejuani,Zilean,Caitlyn,Janna,0.364412,25.691040,7.573643,29.059786,7.444721,25.691040,7.573643,21.148485,7.891978,27.912259,7.270662,22.799783,8.101604,28.538083,7.919328,29.059786,7.444721,30.386495,7.716534,26.415486,8.134784,25.000000,8.333333,25.000000,8.333333
8,TRLH1/1002440150,2018-01-22 01:00:51,Golden Guardians,Gangplank,Jarvan IV,Zoe,Varus,Taric,0,Cloud9,Kled,Evelynn,Ryze,Tristana,Tahm Kench,0.368138,27.032580,7.259700,25.000000,8.333333,27.032580,7.259700,23.426554,7.671105,25.000000,8.333333,21.746319,7.927107,24.809719,7.271232,25.000000,8.333333,26.834839,8.114868,25.798262,7.583816,30.782073,7.614664,23.046669,8.102327
9,TRLH1/1002440161,2018-01-22 02:02:56,Echo Fox,Gnar,Zac,Azir,Ezreal,Taric,1,Clutch Gaming,Ornn,Sejuani,Ryze,Tristana,Br

### Calculate Future Winner from Input Teams

Team1 and Team2 input tuples for win_probability function  

(trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333))  

(trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333))

In [21]:
print(champion_ratings_dict)

{'Alistar': trueskill.Rating(mu=25.407, sigma=2.258), 'Gangplank': trueskill.Rating(mu=30.971, sigma=2.048), 'Gnar': trueskill.Rating(mu=20.287, sigma=2.894), 'Gragas': trueskill.Rating(mu=27.609, sigma=2.134), 'Kalista': trueskill.Rating(mu=24.545, sigma=2.556), 'Malzahar': trueskill.Rating(mu=36.047, sigma=4.561), 'Ryze': trueskill.Rating(mu=22.751, sigma=2.372), 'Shyvana': trueskill.Rating(mu=22.127, sigma=7.966), 'Taric': trueskill.Rating(mu=25.646, sigma=3.575), 'Tristana': trueskill.Rating(mu=21.367, sigma=3.382), 'Braum': trueskill.Rating(mu=25.672, sigma=1.642), "Cho'Gath": trueskill.Rating(mu=23.701, sigma=3.322), 'Ezreal': trueskill.Rating(mu=24.512, sigma=1.671), "Kog'Maw": trueskill.Rating(mu=24.995, sigma=4.065), 'Morgana': trueskill.Rating(mu=25.443, sigma=2.875), 'Rengar': trueskill.Rating(mu=18.447, sigma=7.384), 'Sejuani': trueskill.Rating(mu=24.246, sigma=1.801), 'Shen': trueskill.Rating(mu=27.755, sigma=2.804), 'Azir': trueskill.Rating(mu=24.470, sigma=2.031), 'Evely

TSM = ['Huni', 'Spica', 'PowerOfEvil', 'Lost', 'SwordArt']
FLY = ['Licorice', 'Josedeodo', 'Palafox', 'Johnsun', 'Diamond']
TSM_RATINGS = []
FLY_RATINGS = []

for player in TSM:
    TSM_RATINGS.append(player_ratings_dict[player])

for player in FLY:
    FLY_RATINGS.append(player_ratings_dict[player])
    
TSM_TUPLE = tuple(TSM_RATINGS)
FLY_TUPLE = tuple(FLY_RATINGS)

team_1_win_prob = win_probability(TSM_TUPLE, FLY_TUPLE, ts)
print('TSM Win Probability :' + str(team_1_win_prob))

In [22]:
#print(analyzed_gameids)

In [25]:
#lcs_rating.to_csv('./lcs_champion_trueskill_2018-2021.csv', index=False)